# VAPr Quick-Start Guide

#### Authors: C. Mazzaferro, A. Birmingham, Kathleen Fisch
#### Contact Email: kfisch@ucsd.edu
#### Last Update: February 2017
 
## Table of Contents

* [Introduction](#Introduction)
* [Installation](#Installation)
    * [Python 3 and MongoDB](#Python-3-and-MongoDB)
    * [ANNOVAR](#ANNOVAR)
    * [VAPr](#VAPr)
    * [Dataset Downloads](#Dataset-Downloads)
* [Usage](#Usage)
    * [Novel Variant Annotation](#Novel-Variant-Annotation)
    * [Known Variant Annotation and Storage](#Known-Variant-Annotation-and-Storage)
    * [Variant Prioritization](#Variant-Prioritization)
    * [File Export](#File-Export)
* [Citations](#Citations)

## Introduction

This notebook walks through the steps of annotating variants from a VCF efficiently and thoroughly using the package VAPr. The package retrieves variant information from [ANNOVAR](http://annovar.openbioinformatics.org/en/latest/) and [myvariant.info](myvariant.info) and consolidates it into a single local database for ease of use in investigating and filtering variant findings.

For a more complete description of the functionalities of the package, you can visit the [VAPr Sample Usage Notebook](https://github.com/ucsd-ccbb/VAPr/blob/master/VAPr%20Sample%20Usage.ipynb) and/or check the full documentation on GitHub. 

*Note to reviewers*: this notebook is in all its functionalities the same as the one in the official documentation (https://github.com/ucsd-ccbb/VAPr/blob/master/VAPr%20Quick-Start%20Guide.ipynb). We have added some notes here regarding performance and regarding the file structure of this instance.

## Installation

VAPr requires a Python 3 environment with the pandas library and MongoDB installed.  Here, we provide instructions for how to set up such an environment using  `conda`, a cross-platform package and environment manager.  (If you wish to follow this approach but do not already have either the full or minimal version of `conda` (`Anaconda` or `miniconda`, respectively) installed on your system, visit [https://conda.io/docs/get-started.html](https://conda.io/docs/get-started.html) to decide which is right for you and then follow the instructions provided there to install it.)

### Python 3 and MongoDB

VAPr is written in Python 3, and stores variant annotations in NoSQL database, using a locally-installed instance of MongoDB.  If you wish to run the Jupyter notebooks provided with VAPr locally, it is also necessary to have a local Jupyter notebook server installed. To set up these requirements in your machine's default environment using `conda`, run the following command:

    conda install python=3 pandas mongodb pymongo jupyter notebook

MongoDB also needs a location to store its data, so create a directory for this in the location of your choice, e.g.:

    mkdir -p /Temp/MongoDbData
    
(or, on Windows systems, `md /Temp/MongoDbData`). 

Then start the MongoDb with the command

    mongod --dbpath /Temp/MongoDbData

    
To check if mongodb is currently running, run:

    service mongod status
    
which should return    
    
    mongod (pid 591) is running...
    
519 is the process number so it may differ every time it is run. In case it is not running, the command will return:

    mongod dead but subsys locked
    
    
### VAPr

VAPr is available from PyPi.  Once the above requirements have been installed, VAPr itself can be installed by just running:

    pip install VAPr


### ANNOVAR
(It is possible to proceed without installing ANNOVAR. In that case, the variants that will be annotated and sent ot Mongo are the ones found in MyVariant.info. In that case, users can skip the next steps and go straight to the section **Known Variant Annotation and Storage**)


Users who wish to annotate novel variants will also need to have a local installation of the popular command-line software ANNOVAR([1](#Citations)), which VAPr wraps with a Python interface.  If you use ANNOVAR's functionality through VAPr, please remember to cite the ANNOVAR publication (see #1 in [Citations](#Citations))!

The base ANNOVAR program must be installed by each user individually, since its license agreement does not permit redistribution.  Please visit the ANNOVAR download form at [http://www.openbioinformatics.org/annovar/annovar_download_form.php](http://www.openbioinformatics.org/annovar/annovar_download_form.php), ensure that you meet the requirements for a free license, and fill out the required form.  You will then receive an email providing a link to the latest ANNOVAR release file.  Download this file (which will usually have a name like `annovar.latest.tar.gz`) and place it in the location on your machine in which you would like the ANNOVAR program and its data to be installed--the entire disk size of the databases will be around 25 GB, so make sure you have such space available!  Record the complete path to this location in the `ANNOVAR_PATH` variable below so that VAPr will know where to access it:


In [ ]:
# NOTE: You MUST fill in this variable with the location of ANNOVAR *on your system* 
ANNOVAR_PATH = # e.g., /User/Me/Applications/annovar/

Note that if execution of the above cell caused a syntax error, you have probably forgotten to fill in the path to ANNOVAR on your system; please add that information to the above cell and run it again!

Then simply extract the ANNOVAR tar.gz file in that location to complete the software installation. 

### Dataset Downloads

ANNOVAR requires a number of external data files containing annotation information.  VAPr provides wrapper functions that automatically download these data files and, when desired, update them. The data sources currently supported by default are:
 
- knownGene
- tfbsConsSites
- cytoBand
- genomicSuperDups
- esp6500siv2_all
- 1000g2015aug_all
- popfreq_all
- clinvar_20140929
- cosmic70
- nci60

These can be expanded or restricted by the user, depending on his or her research needs. 

The simple Python code below downloads and installs the necessary databases for ANNOVAR. It currently supports the three different genome builds, `hg19`, `hg18`, `hg38`


In [3]:
from src import base, parsers
import importlib
importlib.reload(base)
importlib.reload(parsers)

# Directory of input files to be annotated
IN_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_files/small_files/"

# Output file directory
OUT_PATH = "/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_files/small_samples/"

# Location of your annovar download. The folder should contain the following files/directories:
ANNOVAR_PATH = '/Volumes/Carlo_HD1/CCBB/annovar/'  


# Design File (optional)
design_file = '/Volumes/Carlo_HD1/CCBB/VAPr_files/des_file.csv'

# Database and Collection names (optional)
proj_data = {'db_name': 'VariantDatabase',
            'project_name': 'collect'}


Project = base.AnnotationProject(IN_PATH, 
                                 OUT_PATH, 
                                 ANNOVAR_PATH, 
                                 proj_data, 
                                 design_file=design_file, 
                                 build_ver='hg19')

In [4]:
Project.parallel_annotation_and_saving(n_processes=6)

[('Normal_targeted_seq', '/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_files/small_files/Normal_targeted_seq.vcf', '/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_files/small_samples/Normal_targeted_seq_annotated.hg19_multianno.txt'), ('Mutect_Pass_Filtered_tumor', '/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_files/small_files/Mutect_Pass_Filtered_tumor.vcf', '/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_files/small_samples/Mutect_Pass_Filtered_tumor_annotated.hg19_multianno.txt')]
950
querying 1-1...querying 1-1...INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): myvariant.info
INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): myvariant.info
952
querying 1-1...querying 1-1...INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): myvariant.info
INFO:requests.packages.urllib3.connectionpool:Starting new HTTP connection (1): myvariant.info
done.
done.
done.
done.
['chr', 'start', 'end', 'ref', 'alt', 'popfreqmax', '1000g_all', '

ValueError: Unrecognized format string: GT:AD:BQ:DP:FA:SS

['chr', 'start', 'end', 'ref', 'alt', 'popfreqmax', '1000g_all', '1000g_afr', '1000g_amr', '1000g_eas', '1000g_eur', '1000g_sas', 'exac_all', 'exac_afr', 'exac_amr', 'exac_eas', 'exac_fin', 'exac_nfe', 'exac_oth', 'exac_sas', 'esp6500siv2_all', 'esp6500siv2_aa', 'esp6500siv2_ea', 'cg46', 'clinsig', 'clndbn', 'clnacc', 'clndsdb', 'clndsdbid', 'cytoband', 'esp6500siv2_all', 'nci60', 'targetscans', '1000g2015aug_all', 'genomicsuperdups', 'func_knowngene', 'gene_knowngene', 'genedetail_knowngene', 'exonicfunc_knowngene', 'aachange_knowngene', 'tfbsconssites', 'cosmic70', 'otherinfo']


In [93]:
import pandas
pandas.read_csv(design_file).set_index('Sample').T.to_dict()

{'Mutect_Pass_Filtered_tumor': {'Condition': 'BD_lithium_responder',
  'Patient': 'JNJ005',
  'Sample_Name2': 'sample3',
  'Tissue': 'lymphoblast',
  'Treatment': 'Li',
  'libType': 'singleend'},
 'Normal_targeted_seq': {'Condition': 'BD_lithium_responder',
  'Patient': 'JNJ005',
  'Sample_Name2': 'sample2',
  'Tissue': 'lymphoblast',
  'Treatment': 'VPA',
  'libType': 'singleend'}}

In [6]:
Project.parallel_annotation_and_saving(n_processes=6)

{}


### Dowloading the files is usually fairly fast, but uncompressing them takes some time. To check the status, run the next command and make sure all files are in .txt and .idx format. 

In [2]:
import subprocess, os
subprocess.Popen(['open', os.path.join(ANNOVAR_PATH, 'humandb')], stdout=subprocess.PIPE)

## Usage

### Novel Variant Annotation

Variant-calling pipelines often identify novel variants that have not been annotated in existing data sources. VAPr encourages (but does not require) the use of ANNOVAR to assign even these novel variants minimal annotation such as predicted mutation type.  Such primary annotation can be run through VAPr as shown in the below example:

In [ ]:
# Get an estimate of the number of variants in the vcf file
print("Number of variants in vcf file: %i" % sum(1 for line in open(IN_PATH)))

In [ ]:
%%time
Project.run_annovar()

~2 minutes for 26k variants

Note that the screen output of `run_annovar` includes a statement like the following:

    Annovar finished working on file : test_file_one_annotated.A.txt file has been created in the OUT_PATH directory
    
This file name and path will be used in the next step.

### Known Variant Annotation and Storage

Most often, researchers are primarily interested in variants about which some information is already available (such as population frequency or disease association).  VAPr utilizes the MyVariant.info web service ([2](#Citations)) to identify up-to-date annotation information from more than a dozen sources for such known variants; the resulting annotations are then stored to the local MongoDB database.  If additional annotations should be included--such as those generated by ANNOVAR for novel variants--these can be provided as a CSV-formatted file.  

To attach known annotations to variants and store them to the local database, it is only necessary to provide three pieces of information:
 1. The path to the VCF file containing the variants to be annotated and stored
 2. The name of the MongoDB database to which the variants will be stored.  Note that this database need not be created ahead of time by the user: if a database of the input name does not exist, VAPr will automatically create one.
 3. The name of the MongoDB collection to which the variants will be stored.  As for the database, this collection need not pre-exist, since VAPr will automatically create it if necessary.
 
Optionally, the user may also provide the path to a CSV file containing additional annotations to be stored. These are the annotations that would be retrieved by annovar. If the argument `annotated_file=csv_file` is not provided, then the package only gets variants from MyVariant.Info

Below is an example of the simple Python required to annotate and store variants, including novel variant annotations produced earlier with ANNOVAR:

In [1]:
vcf_file = # e.g., '/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_files/not_annotated/Normal_targeted_seq.vcf'
csv_file = # e.g., '/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_files/Normal_targeted_seq_annotated.hg19_multianno.txt'

project_data= {'db_name': 'VariantDatabase', 
               'project_name': 'my_project',  # All variants are stored to a collection named as the project name
               'patient_id': 'patient_1',
               'sample_id': 'sample_1'
               }

pars = parser_models.VariantParsing(vcf_file, project_data, annotated_file=csv_file)

SyntaxError: invalid syntax (<ipython-input-1-45250821d8a6>, line 1)

In [5]:
import importlib
from VAPr import parser_models, annovar_suprocess
importlib.reload(parser_models)

ImportError: No module named 'VAPr'

In [16]:
import importlib
from VAPr import parser_models, annovar_suprocess
importlib.reload(parser_models)

IN_PATH = '/Volumes/Carlo_HD1/CCBB/VAPr_files/vcf_files/not_annotated/'
OUT_PATH = '/Volumes/Carlo_HD1/CCBB/VAPr_files/csv_parallel/'

project_data= {'db_name': 'VariantDatabase', 
               'project_name': 'my_project',  # All variants are stored to a collection named as the project name
               'patient_id': 'patient_1',
               'sample_id': 'sample_1'
               }

pars = parser_models.MultiThreadedParsing(IN_PATH, project_data, OUT_PATH)

In [14]:
pars.parallel_variant_parsing(4)

querying 1-950...done.


ValueError: Unrecognized format string: GT:AD:BQ:DP:FA:SS

querying 1-952...done.
querying 1-950...done.
querying 1-950...done.
querying 1-950...done.
querying 1-960...done.
querying 1-950...done.
querying 1-950...done.
querying 1-952...done.
querying 1-950...done.
querying 1-950...done.
querying 1-957...done.
querying 1-950...done.
querying 1-950...done.
querying 1-950...done.
querying 1-954...done.
querying 1-950...done.
querying 1-950...done.
querying 1-955...done.
querying 1-950...done.
querying 1-950...done.
querying 1-957...done.
querying 1-950...done.
querying 1-950...done.
querying 1-956...done.
querying 1-951...done.
querying 1-961...done.
querying 1-957...done.
querying 1-950...done.
querying 1-953...done.
querying 1-950...done.
querying 1-955...done.
querying 1-950...done.
querying 1-956...done.
querying 1-950...done.
querying 1-952...done.
querying 1-951...done.
querying 1-957...done.
querying 1-951...done.
querying 1-955...done.
querying 1-954...done.
querying 1-950...done.
querying 1-950...done.
querying 1-950...done.
querying 1-

Process ForkPoolWorker-23:


### annotate_and_save


The main method of the package is `annotate_and_save`. It will take care of grabbing your variants from a vcf file, extracting the HGVS IDs, pulling the variants from MyVariant.info and sending them to mongo. 

In [ ]:
%%time 
pars.annotate_and_save()

~3 minutes for 25k variants

A great introduction to the concepts relevant to MongoDB, and specifically how to interact with it using python can be found [here](https://docs.mongodb.com/getting-started/python/introduction/). Of particular interest is the explanation on how documents are formatted and stored inside a the database. Variants will follow that format as well: a sample entry of the database will look roughly as follows (the document for this variant (HGVS_id: chr20:g.25194768A>G) has been reduced to half of its actual size). 


```python
{
  "_id": ObjectId("5887c6d3bc644e51c028971a"),
  "chr": "chr19",
  "cytoband": {
    "Region": "13",
    "Sub_Band": "41",
    "Chromosome": 19,
    "Band": "q",
    "Name": "19q13.41"
  },
  "alt": "C",
  "hg19": {
    "end": 25194768,
    "start": 25194768
  },
  
  ...
  
  "otherinfo": [
    "GT:AD:DP:GQ:PL",
    "1/1:0,2:2:6:89,6,0"
  ],
  "hgvs_id": "chr20:g.25194768A>G",
  "esp6500siv2_all": "0.71",
  "ref": "T",
  "chrom": "20",
  "grasp": {
    "exclusively_male_female": "n",
    "initial_sample_description": [
      "Up to 46186 EA individuals",
      [
        "Up to 3445 EA cases",
        " 6935 EA controls"
      ],
      "69395 EA individuals",
      "69395 EA individuals"
    ],
    "creation_date": "8/17/12",
    "gwas_ancestry_description": "European",
    "srsid": 6083780,
    "platform_snps_passing_qc": [
      "Affymetrix & Illumina [~2.5 million] (imputed)",
      "Illumina [528745]",
      [
        "Affymetrix",
        " Illumina & Perlegen [~2.5 million] (imputed)"
      ],
      [
        "Affymetrix",
        " Illumina & Perlegen [~2.5 million] (imputed)"
      ]
    ],
    "hg19": {
      "chr": 20,
      "pos": 25194768
    },
    "publication": [
      {
        "pmid": 20081858,
        "phenotype": "HOMA-B",
        "p_value": 0.033930000000000001825,
        "snpid": "rs6083780",
        "paper_phenotype_description": [
          "Glucose homeostasis traits (fasting glucose",
          " fasting insulin",
          " HOMA-B",
          " HOMA-IR)"
        ],
        "date_pub": "1/17/2010",
        "title": "New genetic loci implicated in fasting glucose homeostasis and their impact on type 2 diabetes risk.",
        "location_within_paper": "FullData",
        "paper_phenotype_categories": "Quantitative trait(s);Type 2 diabetes (T2D);Blood-related",
        "journal": "Nat Genet"
      },
      {
        "pmid": 20522523,
        "phenotype": "Partial epilepsy",
        "p_value": 0.022849999999999998784,
        "snpid": "rs6083780",
        "paper_phenotype_description": "Epilepsy (partial epilepsy)",
        "date_pub": "6/22/2010",
        "title": "Common genetic variation and susceptibility to partial epilepsies: a genome-wide association study.",
        "location_within_paper": "FullScan",
        "paper_phenotype_categories": "Neuro;Epilepsy",
        "journal": "Brain"
      }
    ],
    "last_curation_date": "8/17/12",
    "replication": [
      {
        "total_samples": 76558,
        "european": 76558
      },
      {
        "total_samples": 133661,
        "european": 133661
      }
    ],
    "discovery": [
      {
        "total_samples": 46186,
        "european": 46186
      },
      {
        "total_samples": 10380,
        "european": 10380
      }
    ],
    "hupfield": "Jan2014",
    "includes_male_female_only_analyses": "n",
    "in_gene": "(ENTPD6)",
    "replication_sample_description": [
      "up to 76558 EA individuals",
      "NR",
      "133661 EA individuals",
      "133661 EA individuals"
    ]
  },
  "start": 51447065
} ```


The richness of the data derives from the usage of MyVariant.info services and the high-availability of the datasets hosted by them. Further, being updated at least monthly, their databases can be guaranteed to deliver the most accurate and relevat data for specific variants. 

## Variant Prioritization

Once annotated, variants can be prioritized for further examination by applying built-in or custom filters.  For example, the built-in filter function `rare_cancer_variant()` will identify variants with annotation that satisfies all of the below filter conditions:

 - filter 1: ThousandGenomeAll < 0.05 or info not available
 - filter 2: ESP6500siv2_all < 0.05 or info not available
 - filter 3: cosmic70 information is present
 - filter 4: Func_knownGene is exonic, splicing, or both
 - filter 5: ExonicFunc_knownGene is not "synonymous SNV"
 - filter 6: Read Depth (DP) > 10
 
Identifying these variants with VAPr requires very little coding.  (Note that in this case, the database name and collection name *do* need to refer to existing items, since they specify the variant data that will be searched).

In [ ]:
from VAPr import MongoDB_querying

collection_name = 'My_Variant_Collection_File_One'
db_name = 'My_Variant_Database'

filter_collection = MongoDB_querying.Filters(db_name, collection_name)
rare_cancer_variants = filter_collection.rare_cancer_variant()

Custom filters can be implemented using the keys of the annotation dictionary and the standard [pymongo query syntax](http://api.mongodb.com/python/current/tutorial.html), developed by MongoDB, as shown below:

In [ ]:
from pymongo import MongoClient

client = MongoClient()
db = getattr(client, db_name)
collection = getattr(db, collection_name)


filtered = collection.find({"$and": [
                                   {"$or": [{"esp6500siv2_all": {"$lt": 0.1}}, {"esp6500siv2_all": {"$exists": False}}]},
                                   {"$or": [{"func_knowngene": "exonic"}, {"func_knowngene": "splicing"}]},
                                   {"genotype.filter_passing_reads_count": {"$gte": 1}},
                                   {"cosmic70": {"$exists": True}},
                                   {"1000g2015aug_all": {"$exists": True}}
                         ]})

as_list = list(filtered)
len(as_list)

The returned annotation for each variant matching a filter is provided as a Python dictionary that can easily be accessed from script.  For example, the full annotation for a single variant might look like this:

In [ ]:
rare_cancer_variants[0]

Accessing a particular element of the annotation, such as determining the genomic environment of the annotated variant, can be accessed with simple Python dictionary syntax:

In [ ]:
rare_cancer_variants[0]["annotype"]

Alternately, the list of annotation dictionaries can be loaded into a data frame in the [pandas](http://pandas.pydata.org/) Python data analysis library and then manipulated with all the numerous options provided by that package:

In [ ]:
df = pd.DataFrame(as_list)
df.columns

In [ ]:
df.head(4)

## File Export

Full or filtered sets of variants and their annotations can be exported to CSV:

In [ ]:
rare_cancer_variants_csv = '/data/out_files/rare_vars.csv'

my_writer = file_writer.FileWriter(db_name, collection_name)
my_writer.generate_annotated_csv(rare_cancer_variants, rare_cancer_variants_csv)

Full sets of variants and their annotations can also be output to VCF using a similar but slightly more complex syntax.  Annotations from VAPr will be added to the INFO field:

In [ ]:
rare_cancer_variants_vcf = "/data/out_files/rare_variants.vcf"
input_vcf_compressed = "/data/vcf_files/test_file_one.vcf.gz"
filtered_variant_list = list(filtered)

my_writer = file_writer.FileWriter(db_name, collection_name)
my_writer.generate_annotated_vcf(filtered_variant_list, input_vcf_compressed, rare_cancer_variants_vcf)

Outputting filtered sets of variants to VCF requires somewhat more work, as re-creating a vcf file from a list of variants requires an index file to the original file from which a filtered version will be built. Nonetheless, this can be achieved using the tabix tool from http://genometoolbox.blogspot.com/2013/11/installing-tabix-on-unix.html.

**Step 1**: Download Tabix:

`tar xvjf tabix-0.2.6.tar.bz2`  
`cd tabix-0.2.6`    
`make`    
`export PATH=$PATH:/path_to_tabix/tabix-0.2.6`   

**Step 2**: create a .vcf.gz file from original vcf:

`bgzip -c file.vcf > file.vcf.gz`


**Step 3**: run tabix on the input vcf file:

`tabix -p vcf file.vcf.gz`

## Citations

1. Wang K, Li M, Hakonarson H. ANNOVAR: functional annotation of genetic variants from high-throughput sequencing data. Nucleic Acids Res. 2010 Sep;38(16):e164. doi: 10.1093/nar/gkq603. PubMed PMID: 20601685; PubMed Central PMCID: PMC2938201.
2. Xin J, Mark A, Afrasiabi C, Tsueng G, Juchler M, Gopal N, Stupp GS, Putman TE, Ainscough BJ, Griffith OL, Torkamani A, Whetzel PL, Mungall CJ, Mooney SD, Su AI, Wu C (2016) High-performance web services for querying gene and variant annotation. Genome Biology 17(1):1-7.